# Robot Code in Detail

Previously, we opened and started to take a look at the `Robot.java` file for our example project, and also cloned our own robot for the 2022 season using git. Now we'll take a look a closer look at the code, as well as the organization, patterns, and design philosophy that underpins it.

## Conceptual Organization

Building and programming a robot can be a complicated task that involves keeping track of a large number of electronics. If we try to handle everything just using the features provided in our `Robot.java` file, we'd quickly see that it becomes unsustainable. 

Instead, to keep our code at least moderately understandable, we can split our code from one giant `Robot` class to several individual classes that each handle a specific task, in line with the [seperation of concerns](https://en.wikipedia.org/wiki/Separation_of_concerns) principle.

The benefits of this are that:
- if something doesn't work, we can more easily find where the problem came from
- someone reading our code (including us in the future) can better understand what the code does
- it's much nicer to look at :)


Luckily, WPILib provides us with such a design through their `"Command-Based" Programming` methodology

For our robot, we can break down the task of `running the robot` into 4 seperate layers, those being

1. getting user input
2. turning input into desired actions
3. executing actions on relevant parts of the robot 
4. making the robot respond physically

Splitting it up this way, we would also need some additional system that ties them together.

A simple hypothetical example would be: if (1) a driver pushes a joystick forward on an Xbox controller, (2) it gives a command to drive the robot. This command then (3) tells the subsystem controlling the motors to spin them a certain way, which then (4) makes the robot go forwards. 

Going even further, we see that this framework can be used to further break down layers 3 and 2 into individual pieces, called `subsystem`s, and `command`s.


<center>
  <img src="../Images/SubsysAndComms.svg" alt = "" width = 700/>
</center>


## Subsystems 

> Subsystems encapsulate lower-level robot hardware (such as motor controllers, sensors, and/or pneumatic actuators), and define the interfaces through which that hardware can be accessed by the rest of the robot code. Subsystems allow users to “hide” the internal complexity of their actual hardware from the rest of their code
>
>\- WPILIB documentation

In essense, subsystems are simple abstraction that group bits of hardware that work to control one core part of the robot (like driving). Then, after creating a subsystem, we can communicate with that subsystem directly instead of managing its individual components.

### Creating a subsystem

as Subsystems are quite general containers, so there's not much that we have to provide. To create a subsystem, we'd create a new class that extends from the `SubsystemBase` class, and optionally provide something to do everytime the command scheduler runs (we'll see this later, but it should be the same freqeuency as `robotPeriodic`)

```java
import edu.wpi.first.wpilibj2.command.SubsystemBase;

public class ExampleSubsystem extends SubsystemBase {
  /** Creates a new ExampleSubsystem. */
  public ExampleSubsystem() {}

  @Override
  public void periodic() {
    // This method will be called once per (~20ms)
  }
}
```

which, if you look at the file in you example project, should be in the `frc\robot\subsystems` folder

### Subsystem Example: Drivetrain
The drivetrain consists of the parts of the robot that move the wheels and drive the robot. 

example: West Coast Drive drivetrain

<center>
  <img src="../Images/Drivetrain.png" alt = "" width = 800/>
</center>

The electronic components for the simple model of the drivetrain above are just those four motors (for the sake of space, the example will just have 1).

A drivetrain subsystem should also have a method that handles the process of moving the motors internally.

So in code, a drivetrain class would look something like: 

```java
import edu.wpi.first.wpilibj2.command.SubsystemBase;
// note this is a 3rd party dependency
import com.ctre.phoenix.motorcontrol.can.TalonFX; 

public class ExDrivetrain extends SubsystemBase {
    // we want to handle the motor controllers only in this 
    //subsystem so they should be private
    private TalonFX motor1;
    private TalonFX motor2;

    public ExDrivetrain() {
        motor1 = new TalonFX(/*motor id*/);
        motor1.configFactoryDefault();
        motor1.setNeutralMode(NeutralMode.Brake);

        motor2 = new TalonFX(/*motor id*/);
        motor2.configFactoryDefault();
        motor2.setNeutralMode(NeutralMode.Brake);
    }

    public void driveWheels(double output1, double output2){
        // tells the motor controllers to run the motors 
        motor1.set(output1);
        motor2.set(output2);
    }
}
```

where the `drive` method serves as the main point of interaction from the outside with the subsystem, while its constructor initalizes all the necessary objects and sets up everything to be run

Now that we have a subsystem, we can interact with that subsystem instead of interacting with individual electronic components, making our code a whole lot more understandable.  

## Inputs

Now that we have made commands that correspond to individual actions and responsibilities, we need some way to know when to schedule commands in response to inputs.

Fortunately, WPILib also provides many ways to get user inputs though outside devices.

For our robots in particular, we only use XBox controllers, so we made our own, even simpler system to work with just those controllers.

<center>
  <img src="../Images/XBoxController.png" alt = "" width = 600/>
</center>

### Users In An FRC Competition

In an FRC competition, there will be two drivers for each robot. For us, these two drivers are called the `Primary` and `Secondary` drivers/players. Under the hood however, these just refer to the particular ports which the controllers are assigned to.

### The Controller Class

For our robot, the inputs are handled by the `Controller` class in the `utils` folder.

<center>
  <img src="../Images/ControllerUtil.png" alt = "" width = 600/>
</center>

To create a binding, we use either the `simpleAxis` method(which represents an input between -1 and 1, like a joystick) or the `simpleButton` method (which represents either on or off, like a button).

We first pass in which controller this refers to, as well as what input to bind.
```java
UserDigital buttonA = Controller.simpleButton(
    Controller.PRIMARY,
    Controller.BUTTON_A
);

UserAnalog leftTrigger = Controller.simpleButton(
    Controller.PRIMARY,
    Controller.AXIS_LY
);
```
Note that the values are static constants defined as integers, this is an example of an `enumeration`, where one particular value or `state` is represented as an integer. This is a kind of simplified version of the `Enums` talked about in the previous section, where the names are implicitly given in the form of variables corresponding to different values, as opposed to being explicitly specified in the body of an Enum class

The `UserDigital` and `UserAnalog` interfaces are the things that represent those individual bindings. To get the input at a particular time, we call their `get` method (and they should return either a boolean or a double)

```java
// userdigital example
boolean isButtonADown = buttonA.get();
// useranalog example
double leftTriggerOut = leftTrigger.get();
```

## Inputs with Subsystems

With these two tools, we can start to create a pretty crude and simple design for how our robot might work by directly adding code into the `Robot.java` file (something we won't actually be doing a lot with, and we'll see why in the next document).

A pretty rudimentary example of our robot instructions could be to have it run forwards based on the Y axis values of the left controller joystick during the teleoperated period, which we can represent as the following method in `Robot`:

```java
// in class Robot
private UserAnalog lyaxis = Controller.simpleAxis(Controller.PRIMARY, Controller.AXIS_LY);
private ExDriveTrain dt = new ExDriveTrain();
// ...
@Override
public void teleopPeriodic(){
    dt.drive(lyaxis.get(), lyaxis.get());
}
```

or perhaps we could've wanted to run our robot forwards if a button was pressed and backwards if it was not:

```java
// in class Robot
UserDigital buttonA = Controller.simpleButton(Controller.PRIMARY,Controller.BUTTON_A);
private ExDriveTrain dt = new ExDriveTrain();
// ...
@Override
public void teleopPeriodic(){
    if (buttonA.get())
        dt.drive(0.5,0.5);
    else
        dt.drive(-0.5,-0.5)
}
```

This method of getting inputs frequently, and then passing their current value is a technique known as `polling`. In a large number of cases (and almost always for our inputs), this is sufficent for what we want to accomplish. 

However, there are times--like when we want to find out when a button is clicked, not just when it's on--where this method is cumbersome. For those times, we'll discuss better, command based alternatives that we can use in those cases in the next document in the sections covering `BooleanEvent`s and `Trigger`s

## Program Organization

Now that we've seen some of the individual parts of our design, we can look at how they fit together (take about why we disregard Main.java)

### Robot.java

(note : remember, us humans aren't aren't that good at thinking of mechanisms as actions ....)

The `Robot` class itself doesn't contain that much in both our Robot codebase as well as in the example project, and this is by design. Because the Robot itself operates off of **time**, and not off of **inputs**, we want to limit the amount of work done in this class. 

All the robot handles are background tasks, such as running the command scheduler (robotPeriodic), and handling phase transitions (teleopInit). The heavy work is done by another class, the `RobotContainer` (which Robot creates in robotInit).

### RobotContainer

the `RobotContainer` class handles the setting up, as well as storage of all the individual subsystems and commands. Generally, subsystems and commands are created in the constructor of the RobotContainer, and it also specifies which commands to run during autonomous.

In addition, the RobotContainer should also creates all of our input bindings, before passing them off to the subsystems and commands

### Constants

Often, we'll have many values that represent specific constants and need to be used throughout our project (example being CAN ids). For those values, we put them in a dedicated Constants class. Since we only need one copy of each, want to access it from everywhere, and don't want to change their values, we'll make them `public static final` fields. 

This can get to be quite long and a bit of an eyesore, so we take advantage of another java feature, and just make them fields within an `interface` instead of a class, which makes them automatically public, static, and final.

### Subfolders

besides the main 3 (4) classes, we want to group all our other classes into meaningful folders (and by extension packages).

Two of these groupings are the `subsystems` and `commands` folders, but we also have in our project a `utils`, or utilities folder containing miscellaneous things that are useful, like our class for managing controllers, and the UserDigital/UserAnalog interfaces